<H3> HM APIs testing </H3>

In [1]:
'''HM internal API tests'''

import requests
import json

base_url = "https://api.hotelmap.com"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer hm_live_lI%Jesi1%JJpinzU^eKzVmPp502D7cfPTK13ChNm&05dRFHrnpNPXQP0XJVU7S7xg3EjmER9FN1"
}

def upcoming_events(year):
    '''
    Upcoming events for a given year
    Args: A year

    API Reference:
    GET /v1/cheesefactory/eventfeeds/{year}
    '''
    endpoint = "/v1/cheesefactory/eventfeeds/" + str(year)
    url = base_url + endpoint
    payload = {}
    
    return requests.get(url, headers=headers, json=payload).json()

    
def venue_search(venue, city, top=0):
    """
    Search for venues by name and city.
    Args:
        venue (str): Name or keyword
        city (str): 
        top (int): select only the top result
        
    API Reference:
    POST /v1/cheesefactory/venues/search
    """
    endpoint = "/v1/cheesefactory/venues/search"
    url = base_url + endpoint
    payload = {
        "search_term": venue,  # keyword or venue name
        "city": city           # city to narrow the search
    }

    # If 'top' is specified, return the first venue found
    if top:
        return requests.post(url, headers=headers, json=payload).json()['data']['venues'][0]
    else:
        return requests.post(url, headers=headers, json=payload).json()


def venue_data(venueid):
    """
    Does what it says    
    Args:
        venueid (str): HM ID
        
    API Reference:
    GET /v1/cheesefactory/venues/{venue_id}
    """
    endpoint = f"/v1/cheesefactory/venues/{venueid}"
    url = base_url + endpoint
    payload = {}  # GET request usually doesn't need payload
    return requests.get(url, headers=headers, json=payload)


def load_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)

    print(type(data))  # Print type for debugging (could be dict or list)
    return data


def save_result(response, filename):
    with open(filename, "w") as f:
        json.dump(response, f, indent=4)  # Pretty-print with indentation
    
    return response

In [6]:
'''Return a list of upcoming events in HM system as a spreadsheet'''

import pandas as pd
from IPython.display import display

year = 2025
events = upcoming_events(year)

rows = []
for event in events:
    e = event.get('event', {})
    rows.append({
        "title": e.get('title'),
        "start_date": e.get('date_start'),
        "date_start": e.get('date_start'),
        "venue_id": e.get('venue_id'),
        "is_venue_id_hotel": e.get('is_venue_id_hotel'),
        "city": e.get("city"),
        "map_key": e.get("map_key")
    })

df = pd.DataFrame(rows)

display(df)
df.to_excel(f"data/{str(year)}_events_hotels.xlsx", index=False)

res = save_result(events[31], f"data/{str(year)}_event_0.json")

,title,start_date,date_start,venue_id,is_venue_id_hotel,city,map_key
0,Google Tech Show,2025-10-03,2025-10-03,None,NO,Las Vegas,MZAVZ
1,IMAX event,2025-11-05,2025-11-05,None,NO,Las Vegas,MBUB5
2,IMAX event,2025-11-10,2025-11-10,None,NO,Las Vegas,M4N3R
3,IMAX,2025-11-02,2025-11-02,None,NO,Las Vegas,MG0AX
4,The Atheist,2025-11-02,2025-11-02,None,NO,Las Vegas,MLBG7
...,...,...,...,...,...,...,...
5479,Scott - do not touch,2025-01-06,2025-01-06,20021,NO,London,MJD7K
5480,"The Championships, Wimbledon 2025",2025-06-30,2025-06-30,480476,NO,London,MF8X6
5481,Vienna,2025-02-26,2025-02-26,479007,NO,Wien,MALXH
5482,CHESS CENTER OF NEW YORK,2025-05-24,2025-05-24,25459617,YES,York,M5PQ6
